XML-> CONLL Domain specific
1. (Optional if data present in CONLL Format) Specify path to XML data, mention laptop vs rest; train vs test
2. The following is performed: first tagging based on aspect and opinion lists- if aspect or opinion is more than a word then it is tagged as B. and I.
3. NOISY EXAMPLES (Plenty misspellings and inconsistencies in tagging are removed through supervision)
4. Data is then outputted in CONLL Format


CONLL-> Joint shared vocab and Joint domain labelled vocab 
1. IMPORTANT: Domain name has to be included in CONLL format 


Joint shared vocab -> Trimmed glove vectors
Joint domain labelled vocab -> Trimmed w2vec and geo vectors
1. 
2.






In [211]:
import xml.etree.ElementTree as ET 
import os
import csv 
import sys
import re
import numpy as np
import string
import pickle

In [551]:
xml_path = "./XML_to_CONLL/xmls/Restaurants_Train.xml"
opinion_path = './XML_to_CONLL/Opinions/train_restaurant'
output_csv_path = "./XML_to_CONLL/Rest_Processed.pkl"

In [552]:
tree = ET.parse(xml_path)
with open(opinion_path,'r') as f: #opinions for dataset
    opinion_list = f.readlines()

In [553]:
root = tree.getroot()

In [554]:
sentence = root[0][0].text.replace(u'\xa0',u' ')
#sentence = sentence.translate(string.punctuation)
sentence

#If aspect split

u'But the staff was so horrible to us.'

In [560]:
#len(root)
len(root[252])
root[252][2]

<Element 'aspectCategories' at 0x7f1a3d315110>

In [218]:
opinions = opinion_list[60]
opinions = re.sub(',','',opinions) #removing commas
ops = re.split('([-+][01])?[\r]?[\n]?', re.sub(',','',opinions)) #splitting 
opin_words = list(map(lambda x: x.strip(), ops[:-1:2])) #trimming opinion words
opin_words, opinions

(['NIL'], 'NIL\r\n')

In [219]:
'_ '.strip()

'_'

In [220]:
x = ["Hello","How-are","you!", "doing ?", "toda-y","-","- "]
temp = []
for w in x:
    if(w not in ['-','- ',' -']):
        temp.extend(w.split('-'))
    

In [221]:
temp
temp = [u'i', u'charge', u'it', u'at', u'night', u'and', u'skip', u'taking', u'the', u'cord', u'with', u'me', u'because', u'of', u'the', u'good', u'battery', u'life.']

In [222]:
temp = map(lambda x: x.encode('ascii','replace'),temp) 
map(lambda x: x.translate(None, string.punctuation),temp)

['i',
 'charge',
 'it',
 'at',
 'night',
 'and',
 'skip',
 'taking',
 'the',
 'cord',
 'with',
 'me',
 'because',
 'of',
 'the',
 'good',
 'battery',
 'life']

In [561]:
#tag_to_id = {start_tag:0,end_tag:-1, "BA":1, "IA":2, "BO":3, "IO":4, "OT":5}

def convert_XML_to_CONLL(xml_path, opinion_path, output_csv_path, domain="Laptop", to_lower = True):
    error_list = []
    resultant = []
    res_sentences = []
    res_tags = []
    tree = ET.parse(xml_path) #xml dataset
    with open(opinion_path,'r') as f: #opinions for dataset
        opinion_list = f.readlines()
    root = tree.getroot() 
    if(domain=="Laptop"):
        opinion_list[2702] = 'highly dissatisfied'
        opinion_list[1404] = 'completely immobile'
    
    for i in range(len(root)):
        
        #1) Get tokenized sentence and internally tokenize it
        sentence = root[i][0].text.replace(u'\xa0',u' ')
        sentence_tokenized = sentence.split()
        internally_tokenized_sen = []
        for word in sentence_tokenized:
            word = word.strip()
            if(to_lower):
                word  = word.lower()
            if(word not in ['-']):
                internally_tokenized_sen.extend(word.split('-'))
        sentence_tokenized = internally_tokenized_sen
        #print(sentence_tokenized)
        sentence_tokenized = map(lambda x: x.encode('ascii','replace'), sentence_tokenized)
        sentence_tokenized = map(lambda x: x.translate(None, string.punctuation), sentence_tokenized)
        #sentence_id = root[i].attrib['id']
        tags = ['OT' for word_ in sentence_tokenized]
        
        #2) Get opinion words -> internally tokenized
        opinions = opinion_list[i]
        opinions = re.sub(',','',opinions) #removing commas
        ops = re.split('([-+][01])?[\r]?[\n]?', re.sub(',','',opinions)) #splitting 
        opin_words = list(map(lambda x: x.strip(), ops[:-1:2])) #trimming opinion words
        internally_tokenized_opins = []
        #opin_words = list(map(lambda x:x.split(),opin_words))
        for word in opin_words:
            word = word.strip()
            if(word in ['NIL']):
                continue
            if(to_lower):
                word  = word.lower()
            if(word not in ['-']): #we don't want to include a hyphen
                if("-" in word):
                    internally_tokenized_opins.extend(word.split('-'))
                elif("n't" in word):
                    splits = word.split("n't")
                    splits.remove("")
                    splits = map(lambda x:x.strip(), splits)
                    splits_f = []
                    for w in splits:
                        splits_f.extend(w.split())
                    internally_tokenized_opins.extend(splits_f)
                else:
                    internally_tokenized_opins.extend(word.split())
                
        opin_tokenized = internally_tokenized_opins
        try:
            opin_tokenized = map(lambda x: x.encode('ascii','ignore'), opin_tokenized)
        except:
            print(opin_tokenized,i)
            
        opin_tokenized = map(lambda x: x.translate(None, string.punctuation), opin_tokenized)
        
        
        #3) Get aspect words --> internally tokenized
        aspects = []
        
        if(len(root[i])>1): #aspect words exist
            aspects = []
            for j in range(len(root[i][1])):
                aspects.append(root[i][1][j].attrib['term'].replace(u'\xa0',u' '))
        internally_tokenized_aspects = []
        final_internally_tokenized_aspects = []
        aspects_tokenized = []
        for word in aspects:
            word = word.strip()
            if(to_lower):
                word  = word.lower()
            if(word not in ['-']): #we don't want to include a hyphen
                if("-" in word):
                    internally_tokenized_aspects.extend(word.split('-'))
                elif("n't" in word):
                    splits = word.split("n't")
                    splits.remove("")
                    splits = map(lambda x:x.strip(), splits)
                    splits_f = []
                    for w in splits:
                        splits_f.extend(w.split())
                    internally_tokenized_aspects.extend(splits_f)
                else:
                    internally_tokenized_aspects.extend(word.split())
        for word in internally_tokenized_aspects:
            final_internally_tokenized_aspects.extend(word.split())  
        aspects_tokenized = final_internally_tokenized_aspects
        aspects_tokenized = map(lambda x: x.encode('ascii','replace'), aspects_tokenized)
        aspects_tokenized = map(lambda x:x.translate(None, string.punctuation), aspects_tokenized)
        
    
        #4) Do Process
        #Process
        #Go through each word in sentence tokenized
        #Check if direct match for aspect/opinion
            #If yes then change tag at that position
            #2.2) If previous seq term was B-> then assign I
            #remove the aspect/opinion term
            #assert that aspect and opinion terms list is empty else there was an error
        
        for loc,word in enumerate(sentence_tokenized):
            if(word in aspects_tokenized):
                if(loc>0 and (tags[loc-1]=='BA' or tags[loc-1]=='IA')):
                    tags[loc] = 'IA'
                else:
                    tags[loc] = 'BA'
                aspects_tokenized.remove(word)
                
            elif(word in opin_tokenized):
                if(loc>0 and (tags[loc-1]=='BO' or tags[loc-1]=='IO')):
                    tags[loc] = 'IO'
                else:
                    tags[loc] = 'BO'
                opin_tokenized.remove(word)
                
        if(len(aspects_tokenized)>0 or (len(opin_tokenized)>0 and opin_tokenized!=['nil'] )):
            #Error case-> mostly due to misspellings
            #print(opinions)
            
            print(i, opin_tokenized, sentence_tokenized)
            #print(sentence_tokenized)
            #print("Error at: ",i)
            opin_tokenized_front = []
            opin_tokenized_back  = []
            for op in opin_tokenized:
                opin_tokenized_front.append(op[:len(op)/2]) #CHeck for misspellings
                opin_tokenized_back.append(op[len(op)/2:])
            for loc_i,word in enumerate(sentence_tokenized):
                loc_j = 0 
                for opin_front,opin_back in zip(opin_tokenized_front,opin_tokenized_back):
                    if(opin_front in word):
                        print(i, opin_front, word)
                        x = int(input("Change word?"))
                        if(x==1):
                            word = opin_tokenized[loc_j]
                            if(loc_i>0 and tags[loc_i-1]=='BO'):
                                tags[loc_i] = 'IO'
                            else:
                                tags[loc_i] = 'BO'
                    elif(opin_back in word):
                        print(i, opin_back,word)
                        x = int(input("Change word?"))
                        if(x==1):
                            word = opin_tokenized[loc_j]
                            if(loc_i>0 and tags[loc_i-1]=='BO'):
                                tags[loc_i] = 'IO'
                            else:
                                tags[loc_i] = 'BO'
                    loc_j+=1
                
                        
                 
            error_list.append(i)
            
        resultant.append([sentence_tokenized, tags]) 
        
        
    #Save as csv
    
    
    #Output as CONLL format
    return error_list, resultant
    

In [562]:
z = convert_XML_to_CONLL(xml_path, opinion_path, '.', "Rest")

(52, ['amazing'], ['its', 'a', 'perfect', 'place', 'to', 'have', 'a', 'amanzing', 'indian', 'food'])
(52, 'ama', 'amanzing')
Change word?1
(72, ['like'], ['i', 'liked', 'the', 'beer', 'selection'])
(72, 'li', 'liked')
Change word?1
(73, ['unpretentious'], ['great', 'food', 'good', 'size', 'menu', 'great', 'service', 'and', 'an', 'unpretensious', 'setting'])
(73, 'unpret', 'unpretensious')
Change word?1
(104, ['impeccable'], ['the', 'atmosphere', 'is', 'unheralded', 'the', 'service', 'impecible', 'and', 'the', 'food', 'magnificant'])
(104, 'impec', 'impecible')
Change word?1
(141, ['recomment'], ['the', 'lava', 'cake', 'dessert', 'was', 'incredible', 'and', 'i', 'recommend', 'it'])
(141, 'reco', 'recommend')
Change word?1
(190, ['odd'], ['first', 'off', 'the', 'waitress', 'was', 'completely', 'unattentive', 'the', '2', 'times', 'we', 'saw', 'herodd', 'in', 'a', 'restaurant', 'with', '6', 'tables', 'and', 'got', 'our', 'order', 'wrong'])
(190, 'o', 'off')
Change word?0
(190, 'o', 'comple

In [565]:
len(z[1])

3044

In [385]:
root[0], len(z[1])

(<Element 'sentence' at 0x7f19f166ba10>, 47)

In [625]:
errors= []
for i in range(len(z[1])):
    aspects_tokenized = []
    if(len(root[i])>1): # no aspect word
        aspects = []
        for j in range(len(root[i][1])):
            try:
                aspects.append(root[i][1][j].attrib['term'].replace(u'\xa0',u' '))
            except:
                print(root[i][1][j])
        internally_tokenized_aspects = []
        final_internally_tokenized_aspects = []
        aspects_tokenized = []
        for word in aspects:
            word = word.strip()
            if(True):
                word  = word.lower()
            if(word not in ['-']): #we don't want to include a hyphen
                if("-" in word):
                    internally_tokenized_aspects.extend(word.split('-'))
                elif("n't" in word):
                    splits = word.split("n't")
                    splits.remove("")
                    splits = map(lambda x:x.strip(), splits)
                    splits_f = []
                    for w in splits:
                        splits_f.extend(w.split())
                    internally_tokenized_aspects.extend(splits_f)
                else:
                    internally_tokenized_aspects.extend(word.split())
        for word in internally_tokenized_aspects:
            final_internally_tokenized_aspects.extend(word.split())
            
        aspects_tokenized = final_internally_tokenized_aspects
        aspects_tokenized = map(lambda x: x.encode('ascii','replace'), aspects_tokenized)
        aspects_tokenized = map(lambda x:x.translate(None, string.punctuation), aspects_tokenized)
        
    for loc,word in enumerate(z[1][i][0]):
            tags = z[1][i][1]
            if(word in aspects_tokenized):
                if(loc>0 and (tags[loc-1]=='BA' or tags[loc-1]=='IA')):
                    tags[loc] = 'IA'
                else:
                    tags[loc] = 'BA'
                aspects_tokenized.remove(word)
            if(word[:-1] in aspects_tokenized):
                if(loc>0 and (tags[loc-1]=='BA' or tags[loc-1]=='IA')):
                    tags[loc] = 'IA'
                else:
                    tags[loc] = 'BA'
                aspects_tokenized.remove(word[:-1])
                word = word[:-1]
    if(len(aspects_tokenized)>0):
        errors.append([i, aspects_tokenized, z[1][i]])
        print(i,aspects_tokenized, z[1][i][0])
        

<Element 'aspectCategory' at 0x7f1a44279a10>
<Element 'aspectCategory' at 0x7f1a44279c50>
<Element 'aspectCategory' at 0x7f1a3c41b690>
<Element 'aspectCategory' at 0x7f1a3c41bbd0>
<Element 'aspectCategory' at 0x7f19f381d150>
<Element 'aspectCategory' at 0x7f19f381d090>
<Element 'aspectCategory' at 0x7f19f381db90>
<Element 'aspectCategory' at 0x7f19f381d750>
<Element 'aspectCategory' at 0x7f19f381d650>
<Element 'aspectCategory' at 0x7f19f381de90>
<Element 'aspectCategory' at 0x7f19f381db50>
<Element 'aspectCategory' at 0x7f19f381dd10>
<Element 'aspectCategory' at 0x7f19f381d710>
<Element 'aspectCategory' at 0x7f19f381da50>
<Element 'aspectCategory' at 0x7f1a444d5a90>
<Element 'aspectCategory' at 0x7f19f370fa50>
<Element 'aspectCategory' at 0x7f19f370fd50>
<Element 'aspectCategory' at 0x7f19f370fc10>
<Element 'aspectCategory' at 0x7f1a3cc68e90>
<Element 'aspectCategory' at 0x7f1a3cc68990>
<Element 'aspectCategory' at 0x7f1a3cc68110>
<Element 'aspectCategory' at 0x7f1a3d33b690>
<Element '

In [626]:
len(errors)

2

In [627]:
root[41][1][0].attrib['term']

'interior deco'

In [637]:
errors[1]

[2128,
 ['wines'],
 [['if',
   'anyones',
   'has',
   'doubt',
   'of',
   'not',
   'knowing',
   'enough',
   'about',
   'winesplease',
   'check',
   'their',
   'wine',
   'list'],
  ['OT',
   'OT',
   'OT',
   'OT',
   'OT',
   'OT',
   'OT',
   'OT',
   'OT',
   'OT',
   'OT',
   'OT',
   'BA',
   'IA']]]

In [740]:
z[1][2964][1][-2] = 'BO'

In [379]:
#z[1][400][1][8]= "BO" #, opinion_list[400]
#z[1][381][1][4] = "BO"<-- more than that have to change perfect etc
#z[1][415][0][3] = 'easy'
#z[1][415][1][3] = "BO"
#z[1][453][0][6] = "recommend"
#z[1][453][1][6] = "BO"
#z[1][487][0][3] = "recommend"
#z[1][487][1][3] = "BO"
#z[1][497][0] = ['it',
#z[1][497][1] = ['OT' for x in z[1][497][0]]
#z[1][576][1][8] = 'BO'
#z[1][580][0][-3] = 'easy'
#z[1][580][1][-3] = 'BO'
#z[1][591][0][-1] = 'dependable'
#z[1][591][1][-1]= 'BO'
#z[1][834][0] = ['with',

#z[1][834][1] = ['OT' for x in z[1][834][0]]
#z[1][834][1][7] = "BO"

#z[1][910][1][10] ="BO"
#z[1][937][0][21] = 'recommend'
#z[1][937][1][21] ="BO"

#z[1][1337][0][0] = 'comfortable'
#z[1][1337][1][0] = 'BO'

#z[1][1487][1][3] = 'IO'
#z[1][1652][1][3] = 'BO'
#z[1][1690][1][-3] = 'BO'
#z[1][2129][1][4]  = 'BO'
#z[1][2238][0][4] = 'BO'
#z[1][2331][0][-1]="convenient"
#z[1][2331][1][-1] = "BO"
#z[1][2555][1][-7] = "BO"
#z[1][2601][0][-1]  = 'sensitive'
z[1][2940] =[['strengths', 'well', 'shaped', 'weaknesses', 'a', 'bad', 'videocard'],['OT','BO','BA','OT','OT','BO','BA']]

#= ['strengths','well', 'shaped', 'weaknesses','a', 'bad', 'videocard']

In [570]:
z[1][35]

[['faans',
  'got',
  'a',
  'great',
  'concept',
  'but',
  'a',
  'little',
  'rough',
  'on',
  'the',
  'delivery'],
 ['OT', 'OT', 'OT', 'BO', 'OT', 'OT', 'OT', 'OT', 'BO', 'OT', 'OT', 'OT']]

In [742]:
with open("FINAL_Rest_F.pkl",'w') as p1:
    pickle.dump(z[1], p1)
#with open("FINAL_Rest.pkl",'r') as p1:
#    z_f = pickle.load(p1)

In [549]:
z_f[1]

[['i',
  'bought',
  'a',
  'hp',
  'pavilion',
  'dv4',
  '1222nr',
  'laptop',
  'and',
  'have',
  'had',
  'so',
  'many',
  'problems',
  'with',
  'the',
  'computer'],
 ['OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT',
  'OT']]

In [70]:
vocab = pickle.load(open('../vocab_to_id.pkl','r')) 
embeddings = np.load('../Embeddings/Pruned/np_Laptopw2vec_200d_trimmed.npz')['embeddings']

In [78]:
vocab['windows-xp']

KeyError: 'windows-xp'

In [79]:
embeddings[6307]

array([ 1.324371, -0.425684,  2.100479, -1.525809,  0.037301,  0.031082,
       -0.609131, -1.739496,  0.27888 ,  0.191456,  1.396061, -2.66032 ,
       -1.931001, -0.95014 ,  1.991057, -0.005643, -1.512781,  0.857148,
        0.404849, -1.571726, -2.383786,  0.065251, -0.588009, -0.69691 ,
        1.775787, -2.352656,  0.663636,  1.754127, -2.723913, -0.412109,
       -0.478025, -0.680303,  2.41146 , -2.228422,  2.04625 ,  0.243889,
       -1.05797 , -1.611713, -0.041358,  3.959635, -2.1155  ,  0.977933,
       -0.79038 ,  0.510368,  2.162318, -0.124701, -1.901057, -1.17671 ,
       -0.04299 ,  1.838563,  0.115525,  0.231428, -1.176062, -2.213373,
       -1.769518, -2.025613,  0.783039, -1.043833, -1.761105, -0.935893,
       -0.141148,  3.317784,  1.529664,  0.400738, -2.204345, -1.287052,
        0.588606,  1.820596,  0.169412,  0.386641, -3.007184,  1.080352,
       -0.582821, -0.400848,  1.31329 , -1.799943,  1.540255,  1.069606,
       -0.960693, -0.139252,  0.556204, -2.543706, 

In [852]:
import pickle
domain = "Rest"
tr_data_path = "./XML_to_CONLL/Rest_tr_list.pkl"
tr_split = 85
tr_output_path = "./{}_".format(tr_domain)

In [853]:
def tr_pkl_to_CONLL(domain, tr_data_path, tr_split = 85, tr_output_path = "./"):
    output_path_tr= tr_output_path+"{}_training_data.txt".format(domain)
    output_path_dev = tr_output_path+"{}_dev_data.txt".format(domain)
    tr_list = pickle.load(open(tr_data_path,'r'))
    training_last_index = int((tr_split/100.0)*len(tr_list))
    print(training_last_index)
    training_list = tr_list[:training_last_index]
    dev_list = tr_list[training_last_index:]
    
    with open(output_path_tr,'w') as f1:
        for words,tags in training_list:
            for i, word in enumerate(words):
                f1.write(word + " "+ tags[i]+"\n")
            f1.write("\n")
    print("Written output for training file to: {}".format(output_path_tr))
    with open(output_path_dev,'w') as f1:
        for words,tags in dev_list:
            for i, word in enumerate(words):
                f1.write(word + " "+tags[i]+"\n")
            f1.write("\n")
    print("Written output for dev file to: {}".format(output_path_dev))    
    

In [854]:
with open('../old_samples/test.txt') as f1:
    x = f1.readlines()

In [855]:
tr_pkl_to_CONLL(domain, tr_data_path)

2587
Written output for training file to: ./Rest_training_data.txt
Written output for dev file to: ./Rest_dev_data.txt


In [858]:
with open('./Rest_dev_data.txt') as f1:
    x2 = f1.readlines()

In [859]:
x[0:10],x2[0:19]

(['Jean B-PER\n',
  'Pierre I-PER\n',
  'lives O\n',
  'in O\n',
  'New B-LOC\n',
  'York I-LOC\n',
  '. O\n',
  '\n',
  'The O\n',
  'European B-ORG\n'],
 ['over OT\n',
  'time OT\n',
  'the OT\n',
  'food BA\n',
  'quality IA\n',
  'has OT\n',
  'decreased BO\n',
  'substantially OT\n',
  'it OT\n',
  'is OT\n',
  'a OT\n',
  'lot OT\n',
  'less OT\n',
  'crowded BO\n',
  'than OT\n',
  'it OT\n',
  'used OT\n',
  'to OT\n',
  'and OT\n'])

In [812]:
x2[0]

IndexError: list index out of range

In [ ]:
#Now to make a global vocab and trimmed word vectors